In [1]:
import numpy as np

# Exercise 1.1: The way out

In [2]:
M = np.load('maze.npy')

First, we set the diagonal elements of $M$ to zero. $M_{ii}=1$ (as given) would imply that the random walker stays with non-zero probability in the same room within one time step. However, we assume that this does not happen since it would contradict the toy example on the exercise sheet.

In [3]:
for i in range(M.shape[0]):
    M[i,i] = 0

First, we find the shortest way out using the breadth-first search algorithm.

In [4]:
def breadth_search(starting_point, goal, M):
    '''
    Find the shortest way from starting point to goal in the maze specified by M using the breadth first algorithm.
    '''
    parent_dict = {} # store the parents of each visited node in a dictionary to be able to reconstruct the shortes path
    visited = np.zeros(M.shape[0]) # store if room has been visited (0=False, 1=True)
    visited[starting_point] = 1
    queue = [] # store all rooms that still have to be checked
    queue.append(starting_point)
    counter = 0 # index of next element in queue to be chosen
    while counter <= len(queue): 
        curr_room = queue[counter] # select next element from queue to check
        if curr_room == goal:
            break

        neighbours = np.nonzero(M[curr_room,:]) # find all neighbouring rooms of current room
        for neighbour in neighbours[0]:
            # check if room has already been checked; if this is the case, it can be neglected now
            if visited[neighbour] == 0: 
                queue.append(neighbour) # put room into the queue
                parent_dict[neighbour] = curr_room # save the parent room in the dict
                visited[neighbour] = 1 
        counter += 1
        
    if counter > len(queue):
        return 'There is no way out.'
    
    path = [curr_room]
    # reconstruct the path with help of the dictionary
    while curr_room != starting_point:
        curr_room = parent_dict[curr_room]
        path.append(curr_room)
    
    return len(path), list(reversed(path))

In [7]:
steps, shortest_path = breadth_search(0,99,M)
print('minimal path length: ' + str(steps))
print(shortest_path)

minimal path length: 14
[0, 1, 4, 78, 77, 80, 83, 84, 85, 89, 91, 94, 96, 99]


The list above displays the sequence of rooms when taking the shortest way out which consists of 14 rooms including starting and target room

In order to derive transition probabilites from the matrix $M$, we have to normalize each row of $M$. The transition matrix is then given by $P'_{ij}=\frac{M_{ij}}{\sum_{j=0}^{99}M_{ij}}$, where $P'_{ij}$ now denotes the probability to go from room $i$ to room $j$.

In [9]:
P = (M.T / np.maximum(0.001, np.sum(M, axis=1))).T 
# prevent division by zero for rows only containing zeros (=isolated rooms)

Now we want to derive a method to find the most likely between 2 rooms in the maze. For this purpose we firstly observe that the most likely path must consist of at least as many rooms as the shortest path (which can be found using the algorithm above) and maximally 100 rooms (if it would be more rooms, at least one room would have been visited twice and it could no longer be the most probable path).<br>
Now our idea is to use the Viterbi algorithm in order to find the most probable path and the corresponding probability for a fixed path length $N_T$. This can be repeated for all possible path lengths between the shortest and longest (100 rooms) path. Then we select the most probable one of those pathes, which is the overall most probable path to get from room $i$ to $j$.<br>
In order to apply the Viterbi algorithm, we first have to define observation space, state space, initial probabilities, sequence of observations, the transition matrix and the emission matrix, which is done as follows:
<ul>
<li>The state space consists of all $N_K=100$ rooms of the maze and is thus given by $S=\{0,1,2,3,...,98,99\}$</li>
<li>We define the observation space as $O=\{0,1,2\}$, where 0 represents the observation when being in starting room $i$, 1 the observation for the target room $j$ and 2 the observation for any other room in between.</li>
<li>Then, the sequence of observations is given by $Y=(0,\underbrace{2,2,2,2,...,2,2,2}_{N_T-2\mathrm{times}},1)$ because we start in room $i$ ($\hat{=}$ observation 0), end in room $j$ ($\hat{=}$ observation 1) and are in other rooms during all other states ($\hat{=}$ observation 2).</li>
<li>The initial probabilities must obviously be given by a $N_K$-dimensional array $\Pi$ with $\Pi_k=\begin{cases}1&\mathrm{if}\quad k=i\\0&\mathrm{else}\end{cases}$ since we certainly begin in room $i$.</li>
<li>Finally, the transition matrix is simply the transition matrix derived above whereas the emission matrix is a $N_k\times 3$-dimensional matrix $B$, where $B_{kl}$ denotes the probability of observation $O_l$ given state $S_k$. In our case, it is thus given by $B_{k1}=\begin{cases}1&\mathrm{if}\quad k=i\\0&\mathrm{else}\end{cases}$, $B_{k2}=\begin{cases}1&\mathrm{if}\quad k=j\\0&\mathrm{else}\end{cases}$ and $B_{k3}=\begin{cases}0&\mathrm{if}\quad k=i\lor k=j\\1&\mathrm{else}\end{cases}$ since we definitely know that rooms $i$ and $j$ are starting and target room respectively and all other rooms are just intermediate rooms.</li></ul>
This choice of emission matrix $B$ and intial probability $\Pi$ illustrates that our Markov chain is inherently no Hidden Markov Model. But in order to apply the Viterbi algorithm, we had to represent it this way.

In [10]:
def Viterbi(O, S, PI, Y, A, B):
    '''
    Implementation of the Viterbi algorithm using the nomenclature from https://en.wikipedia.org/wiki/Viterbi_algorithm:
    O: observation space
    S: state space
    PI: array of initial probabilites: PI[i] = p(x_1 = S[i])
    Y: sequence of observations: y[i] = j if y[i] = O[j] 
    A: transition matrix
    B: emission matrix: B_{ij} = p(y_j=o_j | x_i)
    '''
    T = len(Y)
    K = len(S)
    X = np.zeros(T)
    T1 = np.zeros([K,T])
    T2 = np.zeros([K,T])
    index = Y[0].astype(int)
    T1[:,0] = PI * B[:,index]
    T2[:,0] = 0
    
    for i in range(1,T):
        for j in range(K):
            index = Y[i].astype(int)
            T1[j,i] = np.max(T1[:,i-1] * A[:,j] * B[j,index])
            T2[j,i] = np.argmax(T1[:,i-1] * A[:,j] * B[j,index])
    
    p = np.max(T1[:,T-1]) # p is the probabilits of the most likely chain
    index = np.argmax(T1[:,T-1])
    X[-1] = S[index] 
    
    for i in range(T-1,0,-1):
        index = T2[index,i].astype(int)
        X[i-1] = S[index]
    
    return p,X

Now, we want to find the most likely way from room 0 to room 99. From above, we know that the shortest path consists of 14 rooms. Thus, we look for the most likely pathes for all path lenghts between 14 and 100 and then select the most likely one of those.

In [11]:
pathes = []
probs = np.zeros(101-14)
for N_T in range(14,101):
    N_K = 100 # number of states
    O = np.array([0,1,2]) # observations space: 0: start, 1: goal, 2: in between 
    S = np.arange(N_K) # state space
    PI = np.zeros(N_K)
    PI[0] = 1 # sequence definitely starts in room 0
    Y = np.zeros(N_T) # sequence of observations
    Y[1:-1] = 2
    Y[-1] = 1
    A = P # transition matrix
    B = np.zeros([N_K, 3]) # emission matrix
    B[0,0] = 1
    B[1:-1,2] = 1
    B[-1,1] = 1
    
    p,X = Viterbi(O, S, PI, Y, A, B)
    probs[N_T-14] = p
    pathes.append(X)
    
most_likely_path = pathes[np.argmax(probs)]
most_likely_path_prob = np.max(probs)
    
print('probability of most likely path: ' + str(most_likely_path_prob))
print('most likely path: ' + str(most_likely_path))

probability of most likely path: 7.65481089555e-09
most likely path: [  0.   1.   4.  78.  79.  80.  83.  84.  85.  89.  91.  94.  96.  99.]


As one can see, the most likely path has the same length as the shortest path in the beginning. However, the most likely path enters room 79 instead of room 77.

In [12]:
def path_prob(X, P):
    '''
    Compute the probability of a given path X and a transitions matrix P
    '''
    room_transitions = [tuple(X[:-1]), tuple(X[1:])]
    p = np.prod(P[room_transitions])
    return p

In [13]:
print('probability of shortest path from above: ' + str(path_prob(shortest_path,P)))

probability of shortest path from above: 6.12384871644e-09


Comparing the probabilites of the shortest and the most likely path, we find that they differ by a factor of $\frac{5}{4}$. This is due to the fact that room 79 has only 4 doors, whereas room 77 has 5 doors. Thus entering room 79, we have a probability of 1/4 to select the "correct" door whereas room 77 only offers a corresponding probability of 1/5.

# Expected traversal time

Following the toy example on the exercise sheet, we can write for a maze consisting of $N$ rooms
$$
h_{ij}=1+\sum_{k=1}^{N}P_{ik}h_{kj}\quad\mathrm{with}\quad h_{ii}=0\forall i,
$$
where $P$ is the transition matrix and $P_{ij}$ denotes the probability to go from room $i$ to room $j$ in a time step. This equation can be arranged in this way for all $i\neq j$. Rearranging terms provides
$$
h_{ij}-\sum_{k=1}^{N}P_{ik}h_{kj}=1\quad\forall i\neq j
$$
which is a linear system of N-1 equations with N-1 unknown variables ($h_{ij}, i=1...N, i\neq j$). Defining the matrix
    $$\bar{P}=\begin{cases}-P_{ij}&\mathrm{if}\quad i\neq j\\1&\mathrm{if}\quad i=j\end{cases}$$
and let $\bar{P}_{\setminus j}$ denote the matrix $\bar{P}$ without column and row with index $j$ (thus a $(N-1)\times (N-1)$-matrix), then the system of equations can be rewritten as
$$\bar{P}_{\setminus j}\cdot\vec{h}=\vec{1}$$
where $\vec{h}_i=h_{ij}$ and $\vec{1}$ a ($N-1$)-dimensional vector of ones.<br>
In principle, this equation can now be solved for $\vec{h}$ yielding the expected traversal time from all rooms to room j. However, in case of the existence of isolated rooms from which one cannot reach the goal room, the matrix will be singular and cannot be inversed. In this case, we either have to remove the corresponding column and row $k$ of the matrix $\bar{P}_{\setminus j}$ if room $k$ is such an isolated room, or we solve the system by minimizing the least squared error.<br>

In the present case, in order to compute $h_{0,99}$ we simply remove the columns and rows of the isolated rooms. Here the isolated rooms are 13, 32 and 90 which do not have any door and rooms 73 and 76 which are only connected to each other.

In [21]:
P_bar = np.zeros(P.shape)
for i in range(99):
    P_bar[i,i]=1
    for j in range(99):
        if i!=j:
            P_bar[i,j] = -P[i,j]
P_bar_lsq = P_bar.copy() # for the least squares solver
P_bar = np.delete(P_bar,(13,32,73,76,90), axis=0)
P_bar = np.delete(P_bar,(13,32,73,76,90), axis=1)
b = np.ones(94)
b_lsq = np.ones(99)

h_099 = np.linalg.solve(P_bar[:-1,:-1],b)[0]
print('analytically expected traversal time from room 0 to 99: ' + str(h_099))

analytically expected traversal time from room 0 to 99: 2202.66393358


For comparison, we also provide the least-squares result which coincides with the solution above:

In [22]:
print('least_squares solution: h_0,99 = ' + str(np.linalg.lstsq(P_bar_lsq[:-1,:-1],b_lsq)[0][0]))

least_squares solution: h_0,99 = 2202.66393358


Now, we want to find the expected traversal time by means of simulating the random walk.

In [23]:
def simulate_RW(start, goal, M):
    '''
    Simulate a random walk in maze specified by M with given start and goal.
    The required amount of steps (=time) will be returned.
    '''
    curr_room = start
    time = 0
    while curr_room != goal:
        neighbours = np.nonzero(M[curr_room,:])[0]
        curr_room = np.random.choice(neighbours)
        time += 1
    
    return time

 <font color="red">das untere nicht noch einmal ausführen, dauert sehr lange!</font> 

In [27]:
N = 100000
escape_times = np.zeros(N)
for i in range(N):
    escape_times[i] = simulate_RW(0,99,M)
print('simulation-based expected traversal time from room 0 to 99: ' + str(np.average(escape_times)))

simulation-based expected traversal time from room 0 to 99: 2203.83144


As one can see, the analytically computed expected traversal time and the average traversal time for 100.000 random walkers coincides very well and deviate by only one time step from each other.